# GISY6400 Capstone - Wildfire Hazard: Fuel Mapping

Program: wildfire_fuel.ipynb  
Programmer: Brian Gauthier  
Purpose: This notebook produces fuel products for wildfire hazard mapping (NDVI, EVI, etc)  
Date: May 5, 2025

### Import Python Modules

In [ ]:
import arcpy
import os
from arcpy.sa import *

### Paths & Workspace Setup

In [ ]:
# Get project directory
aprx = arcpy.mp.ArcGISProject("CURRENT")
gis_dir = os.path.dirname(aprx.filePath)
project_dir = os.path.dirname(gis_dir)

# Directory structure
raw_dir = os.path.join(project_dir, "data", "raw")
extract_dir = os.path.join(project_dir, "data", "extracted")
proc_dir = os.path.join(project_dir, "data", "processed")
land_mask_path = os.path.join(project_dir, "gis", "capstone.gdb", "hydro_la_poly")
sentinel_path = os.path.join(proc_dir, "sentinel_2_mosaic_utm_clipped.tif")

# Create folders if they don't exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(extract_dir, exist_ok=True)
os.makedirs(proc_dir, exist_ok=True)

In [ ]:
# Set ArcPy Workspace & overwrite options
arcpy.env.workspace = os.path.join(project_dir, "gis", "capstone.gdb")
arcpy.env.overwriteOutput = True

arcpy.CheckOutExtension("Spatial")

### Calculate NDVI

In [ ]:
# load multiband raster
sentinel_raster = Raster(sentinel_path)

# Extract Band 8 (NIR) and Band 4 (Red)
nir = Raster(f"{sentinel_path}\\Band_8")
red = Raster(f"{sentinel_path}\\Band_4")

# Compute NDVI
ndvi = (nir - red) / (nir + red)

# Mask NDVI by land area
ndvi_masked = ExtractByMask(ndvi, land_mask_path)

# Save result
ndvi_masked_output = os.path.join(proc_dir, "ndvi.tif")
ndvi_masked.save(ndvi_masked_output)

print(f"NDVI calculation complete. Output saved as {ndvi_masked_output}")

In [ ]:
# Check the range of values

# Open the raster
ndvi_raster = arcpy.sa.Raster(ndvi_masked_output)

# Get the minimum and maximum values of the NDVI raster
min_value = ndvi_raster.minimum
max_value = ndvi_raster.maximum

print(f"Minimum NDVI value: {min_value}")
print(f"Maximum NDVI value: {max_value}")

### Reclassify NDVI

In [ ]:
# Define the reclassification ranges and values
reclass_ranges = [
    (-0.254341, 0, 0),   # Null
    (0, 0.1, 1),         # Very Low (Bare soil/rock)
    (0.1, 0.3, 2),       # Low (Sparse Vegetation/grass)
    (0.3, 0.45, 3),       # Moderate (Moderate Vegetation)
    (0.45, 0.62721, 4)    # High (Dense Vegetation)
]

# Create a Reclassification function
reclass_list = []
for low, high, new_value in reclass_ranges:
    reclass_list.append([low, high, new_value])

# Reclassify NDVI
ndvi_reclass = Reclassify(ndvi_masked, "Value", RemapRange(reclass_list))

# Save the reclassified raster
ndvi_reclass_path = os.path.join(arcpy.env.workspace, "ndvi_masked_reclass")
ndvi_reclass.save(ndvi_reclass_path)

print(f"NDVI reclassification complete. Output saved as {ndvi_reclass_path}")

### Calculate EVI

In [ ]:
# Constants used in the EVI formula
G = 2.5   # Gain factor (sensistivity of the index to the vegetation)
C1 = 6.0  # Correction factor to reduce the effect of red light scattering in the atmosphere
C2 = 7.5  # Correction factor to reduce the effect of blue light scattering (aerosols)
L = 1.0  # Small constant to prevent division by zero and reduce noise in low vegetation areas
scale_factor = 10000.0 # Scale factor for sentinel-2 reflectance

# Load individual bands from the Sentinel-2 mosaic
blue = Raster(f"{sentinel_path}\\Band_2") / scale_factor  # Blue band (used for aerosol resistance)
red = Raster(f"{sentinel_path}\\Band_4") / scale_factor  # Red band (used for vegetation contrast)
nir = Raster(f"{sentinel_path}\\Band_8") / scale_factor  # Near-infrared band (strongly reflected by vegetation)

# Compute the Enhanced Vegetation Index (EVI)
evi = G * ((nir - red) / (nir + C1 * red - C2 * blue + L))

# Mask EVI to limit values to land area only
evi_masked = ExtractByMask(evi, land_mask_path)

# Clip extreme EVI values to avoid values outside the valid range (-1 to 1)
# The problem was here, using `evi_masked` as raster data in Con properly
# Replace values outside [-1, 1] with NoData
evi_clipped = Con(((evi_masked >= -1) & (evi_masked <= 1)), evi_masked)


# Define output path for the masked EVI raster
evi_clipped_output = os.path.join(proc_dir, "evi_clipped.tif")
evi_clipped.save(evi_clipped_output)

print(f"EVI calculation complete. Output saved as {evi_clipped_output}")

In [ ]:
# Check the range of values

# Open the raster
evi_raster = arcpy.sa.Raster(evi_clipped_output)

# Get the minimum and maximum values of the EVI raster
min_value = evi_raster.minimum
max_value = evi_raster.maximum

print(f"Minimum EVI value: {min_value}")
print(f"Maximum EVI value: {max_value}")

### Reclassify EVI

In [ ]:
# Reclassification ranges for EVI values
evi_reclass_ranges = [
    (-1.0, 0.0, 0),     # Class 0: Water or non-vegetated surfaces (low EVI)
    (0.0, 0.2, 1),      # Class 1: Very low vegetation (bare soil/urban)
    (0.2, 0.4, 2),      # Class 2: Low vegetation (sparse vegetation)
    (0.4, 0.6, 3),      # Class 3: Moderate vegetation (moderate vegetation)
    (0.6, 1.0, 4)       # Class 4: High vegetation (dense vegetation)
]


# Convert ranges to the format required by RemapRange
evi_reclass_list = [[low, high, value] for (low, high, value) in evi_reclass_ranges]

# Reclassify the masked EVI raster into vegetation density classes
evi_reclass = Reclassify(evi_clipped, "Value", RemapRange(evi_reclass_list))

# Define output path for the reclassified EVI raster
evi_reclass_path = os.path.join(arcpy.env.workspace, "evi_masked_reclass")
evi_reclass.save(evi_reclass_path)

print(f"EVI reclassification complete. Output saved as {evi_reclass_path}")

### Average NDVI & EVI

In [ ]:
ndvi = Raster(r"D:\Dropbox\COGS\Capstone\gis\capstone.gdb\ndvi_final")
evi = Raster(r"D:\Dropbox\COGS\Capstone\gis\capstone.gdb\evi_final3")

# Average NDVI and EVI
fuel_index = (ndvi + evi) / 2

# Save the combined fuel raster
fuel_index.save(r"D:\Dropbox\COGS\Capstone\gis\capstone.gdb\fuel_index")